In [32]:
import pandas as pd
import catboost as cb
import numpy as np
import gensim.downloader as api
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, hamming_loss
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from datasets_utils import get_luxury_data, get_tech_data, get_retail_data, get_big_basket_data
from preprocess import preprocess, with_category_features

In [2]:
luxury_data = get_luxury_data()
tech_data = get_tech_data()
retail_data = get_retail_data()
big_basket_data = get_big_basket_data()

datasets = [big_basket_data, retail_data, luxury_data, tech_data]
dataset_names = ['Big basket', 'Retail', 'Luxury', 'Tech']

# Get datasets with description column preprocessed
tech_data['description'] = tech_data['description'].apply(preprocess)
luxury_data['description'] = luxury_data['description'].apply(preprocess)
retail_data['description'] = retail_data['description'].apply(preprocess)
big_basket_data['description'] = big_basket_data['description'].apply(preprocess)

# Preprocess categories
tech_data = with_category_features(tech_data)
luxury_data = with_category_features(luxury_data)
retail_data = with_category_features(retail_data)
big_basket_data = with_category_features(big_basket_data)

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [3]:
big_basket_X_train, big_basket_X_test, big_basket_y_train, big_basket_y_test = train_test_split(
    big_basket_data['description'],
    big_basket_data[[column for column in big_basket_data.columns if column != 'description']],
    test_size=0.2,
    random_state=13
)

In [4]:
def tfidf_vectorize(train_data, test_data):
    tfidf = TfidfVectorizer()
    tfidf_train_data = tfidf.fit_transform(train_data)
    tfidf_test_data = tfidf.transform(test_data)
    return tfidf_train_data, tfidf_test_data

In [5]:
big_basket_X_train_tfidf, big_basket_X_test_tfidf = tfidf_vectorize(
    train_data=big_basket_X_train,
    test_data=big_basket_X_test,
)

In [6]:
model = cb.CatBoostClassifier(
    iterations=100,
    loss_function='MultiLogloss',
    class_names=big_basket_y_train.columns.values,
    allow_const_label=True,
    random_seed=13,
)

big_basket_tfidf_pool = cb.Pool(big_basket_X_train_tfidf, big_basket_y_train)

model.fit(big_basket_tfidf_pool, verbose=50)

Learning rate set to 0.318232
0:	learn: 0.2331716	total: 11.4s	remaining: 18m 52s
50:	learn: 0.0357299	total: 9m 56s	remaining: 9m 32s
99:	learn: 0.0270354	total: 19m 25s	remaining: 0us


In [8]:
big_basket_y_pred_tfidf = model.predict(big_basket_X_test_tfidf)

In [24]:
print(f"Accuracy: {accuracy_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf)}")
print(f"Precision (macro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='macro')}")
print(f"Precision (micro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='micro')}")
print(f"Recall (macro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='macro')}")
print(f"Recall (micro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf, average='micro')}")
print(f"Hamming loss: {hamming_loss(big_basket_y_test.to_numpy(), big_basket_y_pred_tfidf)}")
# ConfusionMatrixDisplay(confusion_matrix(big_basket_y_test.to_numpy().argmax(axis=1), big_basket_y_pred_tfidf.argmax(axis=1))).plot()

Accuracy: 0.4165451895043732
Precision (macro): 0.7520990845943957
Precision (micro): 0.8988735919899875
Recall (macro): 0.34214281747008646
Recall (micro): 0.5756191392161577
Hamming loss: 0.01069291040592061


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
model_default = cb.CatBoostClassifier(
    iterations=100,
    loss_function='MultiLogloss',
    class_names=big_basket_y_train.columns.values,
    allow_const_label=True,
    random_seed=13,
)

big_basket_train_pool = cb.Pool(pd.DataFrame(big_basket_X_train.values, columns=['description']), big_basket_y_train.to_numpy(), text_features=['description'], feature_names=['description'])

model_default.fit(big_basket_train_pool, verbose=50)

Learning rate set to 0.318232
0:	learn: 0.4466780	total: 6.18s	remaining: 10m 11s
50:	learn: 0.0282392	total: 6m 5s	remaining: 5m 51s
99:	learn: 0.0229303	total: 11m 31s	remaining: 0us


In [27]:
big_basket_y_pred_default = model_default.predict(pd.DataFrame(big_basket_X_test.values, columns=['description']))

In [29]:
print(f"Accuracy: {accuracy_score(big_basket_y_test.to_numpy(), big_basket_y_pred_default)}")
print(f"Precision (macro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_default, average='macro')}")
print(f"Precision (micro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_default, average='micro')}")
print(f"Recall (macro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_default, average='macro')}")
print(f"Recall (micro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_default, average='micro')}")
print(f"Hamming loss: {hamming_loss(big_basket_y_test.to_numpy(), big_basket_y_pred_default)}")

Accuracy: 0.6065962099125365
Precision (macro): 0.7169155947747387
Precision (micro): 0.8803348050036792
Recall (macro): 0.5097136742134956
Recall (micro): 0.7670914482648072
Hamming loss: 0.007370977237048665


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
wv = api.load('word2vec-google-news-300')

In [35]:
def to_w2v_embedding(sentence):
    embeddings = []

    for word in sentence.split():
        if word in wv:
            embeddings.append(wv[word])
    embeddings = np.array(embeddings)
    return np.mean(embeddings, axis=0)

In [36]:
big_basket_X_train_w2v = big_basket_X_train.apply(to_w2v_embedding)
big_basket_X_test_w2v = big_basket_X_test.apply(to_w2v_embedding)

In [42]:
model_w2v = cb.CatBoostClassifier(
    iterations=100,
    loss_function='MultiLogloss',
    class_names=big_basket_y_train.columns.values,
    allow_const_label=True,
    random_seed=13,
)

big_basket_w2v_pool = cb.Pool(np.array([x for x in big_basket_X_train_w2v]), big_basket_y_train)

model_w2v.fit(big_basket_w2v_pool, verbose=10)

Learning rate set to 0.318232
0:	learn: 0.2148468	total: 2.42s	remaining: 3m 59s
10:	learn: 0.0420730	total: 27.7s	remaining: 3m 43s
20:	learn: 0.0309566	total: 52.1s	remaining: 3m 16s
30:	learn: 0.0249542	total: 1m 16s	remaining: 2m 50s
40:	learn: 0.0207562	total: 1m 40s	remaining: 2m 24s
50:	learn: 0.0177137	total: 2m 4s	remaining: 1m 59s
60:	learn: 0.0153369	total: 2m 29s	remaining: 1m 35s
70:	learn: 0.0136097	total: 2m 53s	remaining: 1m 10s
80:	learn: 0.0120163	total: 3m 16s	remaining: 46.2s
90:	learn: 0.0107490	total: 3m 40s	remaining: 21.9s
99:	learn: 0.0097063	total: 4m 2s	remaining: 0us


In [43]:
big_basket_y_pred_w2v = model_w2v.predict(np.array([x for x in big_basket_X_test_w2v]))

In [44]:
print(f"Accuracy: {accuracy_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v)}")
print(f"Precision (macro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='macro')}")
print(f"Precision (micro): {precision_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='micro')}")
print(f"Recall (macro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='macro')}")
print(f"Recall (micro): {recall_score(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v, average='micro')}")
print(f"Hamming loss: {hamming_loss(big_basket_y_test.to_numpy(), big_basket_y_pred_w2v)}")

Accuracy: 0.5134839650145773
Precision (macro): 0.7595887235355991
Precision (micro): 0.8722147372784744
Recall (macro): 0.4784695681525681
Recall (micro): 0.6745211188587
Hamming loss: 0.009275482170890335


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
